### **Task 2 - Algoritmo Forward Backward en HMM**

In [2]:
import random

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states  # Estados del modelo
        self.observations = observations  # Observaciones posibles
        self.initial_prob = initial_prob  # Probabilidad inicial de los estados
        self.transition_prob = transition_prob  # Probabilidad de transición entre estados
        self.emission_prob = emission_prob  # Probabilidad de emisión de observaciones dado un estado

    def generate_sequence(self, length):
        sequence = []  # Secuencia de observaciones generadas
        state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states], k=1)[0]
        for _ in range(length):
            observation = random.choices(self.observations, weights=[self.emission_prob[state][o] for o in self.observations], k=1)[0]
            sequence.append((state, observation))
            state = random.choices(self.states, weights=[self.transition_prob[state][s] for s in self.states], k=1)[0]
        return sequence

    def forward(self, sequence):
        forward_prob = [{}]  # Probabilidades forward para cada paso de tiempo
        for i, (state, observation) in enumerate(sequence):
            if i == 0:
                forward_prob[i] = {s: self.initial_prob[s] * self.emission_prob[s][observation] for s in self.states}
            else:
                forward_prob.append({})
                for s in self.states:
                    forward_prob[i][s] = sum(forward_prob[i - 1][prev_s] * self.transition_prob[prev_s][s] * self.emission_prob[s][observation] for prev_s in self.states)
        return forward_prob

    def backward(self, sequence):
        N = len(sequence)
        backward_prob = [{} for _ in range(N)]  # Probabilidades backward para cada paso de tiempo
        for i in reversed(range(N)):
            for s in self.states:
                if i == N - 1:
                    backward_prob[i][s] = 1
                else:
                    backward_prob[i][s] = sum(self.transition_prob[s][next_s] * self.emission_prob[next_s][sequence[i + 1][1]] * backward_prob[i + 1][next_s] for next_s in self.states)
        return backward_prob

    def compute_state_probabilities(self, sequence):
        forward_prob = self.forward(sequence)
        backward_prob = self.backward(sequence)
        state_probabilities = []
        for i in range(len(sequence)):
            prob_sum = sum(forward_prob[i][s] * backward_prob[i][s] for s in self.states)
            state_probabilities.append({s: (forward_prob[i][s] * backward_prob[i][s]) / prob_sum for s in self.states})
        return state_probabilities

# Parámetros del HMM
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}

# instancia de HMM
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar una secuencia de observaciones
sequence = hmm.generate_sequence(5)
print(f"Secuencia de observaciones: {sequence}")

# Calcular probabilidades forward
fwd_probs = hmm.forward(sequence)
print(f"Probabilidades forward: {fwd_probs}")

# Calcular probabilidades backward
bwd_probs = hmm.backward(sequence)
print(f"Probabilidades backward: {bwd_probs}")

# Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities(sequence)
print(f"Probabilidades de estados: {state_probs}")


Secuencia de observaciones: [('Sunny', 'Rainy'), ('Sunny', 'Sunny'), ('Sunny', 'Sunny'), ('Sunny', 'Sunny'), ('Sunny', 'Sunny')]
Probabilidades forward: [{'Sunny': 0.1, 'Rainy': 0.35}, {'Sunny': 0.176, 'Rainy': 0.069}, {'Sunny': 0.13472, 'Rainy': 0.02298}, {'Sunny': 0.09357440000000002, 'Rainy': 0.0122196}, {'Sunny': 0.06379788800000001, 'Rainy': 0.007813992}]
Probabilidades backward: [{'Sunny': 0.2204152000000002, 'Rainy': 0.1416296000000001}, {'Sunny': 0.3247600000000002, 'Rainy': 0.2094800000000001}, {'Sunny': 0.4780000000000002, 'Rainy': 0.31400000000000006}, {'Sunny': 0.7000000000000002, 'Rainy': 0.5}, {'Sunny': 1, 'Rainy': 1}]
Probabilidades de estados: [{'Sunny': 0.30779138880308693, 'Rainy': 0.692208611196913}, {'Sunny': 0.7981603052454426, 'Rainy': 0.20183969475455749}, {'Sunny': 0.8992385062366747, 'Rainy': 0.10076149376332527}, {'Sunny': 0.9146817539212768, 'Rainy': 0.08531824607872322}, {'Sunny': 0.8908841382184073, 'Rainy': 0.10911586178159265}]
